In [3]:
from jmetal.algorithm.multiobjective import NSGAII
from jmetal.operator import SBXCrossover, PolynomialMutation
from jmetal.problem import ZDT1
from jmetal.util.termination_criterion import StoppingByEvaluations

problem = ZDT1()

algorithm = NSGAII(
    problem=problem,
    population_size=100,
    offspring_population_size=100,
    mutation=PolynomialMutation(probability=1.0 / problem.number_of_variables(), distribution_index=20),
    crossover=SBXCrossover(probability=1.0, distribution_index=20),
    termination_criterion=StoppingByEvaluations(max_evaluations=25000)
)

algorithm.run()

[2024-04-01 01:39:22,943] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2024-04-01 01:39:22,944] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2024-04-01 01:39:22,945] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2024-04-01 01:39:22,945] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2024-04-01 01:39:26,190] [jmetal.core.algorithm] [DEBUG] Finished!


In [7]:
from jmetal.util.solution import get_non_dominated_solutions, print_function_values_to_file, print_variables_to_file

front = get_non_dominated_solutions(algorithm.get_result())

# save to files
print_function_values_to_file(front, 'FUN.NSGAII.ZDT1')
print_variables_to_file(front, 'VAR.NSGAII.ZDT1')

[2024-04-01 01:43:34,443] [jmetal.util.solution] [INFO] Output file (function values): FUN.NSGAII.ZDT1
[2024-04-01 01:43:34,444] [jmetal.util.solution] [INFO] Output file (variables): VAR.NSGAII.ZDT1


In [8]:
from jmetal.lab.visualization import Plot

plot_front = Plot(title='Pareto front approximation', axis_labels=['x', 'y'])
plot_front.plot(front, label='NSGAII-ZDT1', filename='NSGAII-ZDT1', format='png')

[2024-04-01 01:43:38,229] [jmetal.lab.visualization.plotting] [INFO] Figure {_filename} saved to file


In [ ]:
from jmetal.lab.experiment import Experiment, Job


def experiment_configuration(problems: list, 
                             n_run: int, 
                             n_population_size: int, 
                             ):
 jobs = []

 for run in range(n_run):
     for problem in problems:
         jobs.append(
             Job(
                 algorithm=NSGAII(
                     problem=problem,
                     population_size=100,
                     mating_pool_size=100,
                     offspring_size=100,
                     max_evaluations=5000,
                     mutation=NullMutation(),
                     crossover=SBX(probability=1.0, distribution_index=20),
                     selection=BinaryTournamentSelection(comparator=RankingAndCrowdingDistanceComparator())
                 ),
                 label='NSGAII with Null Mutation',
                 run=run,
             )
         )
         jobs.append(
             Job(
                 algorithm=NSGAII(
                     problem=problem,
                     population_size=100,
                     mating_pool_size=100,
                     offspring_size=100,
                     max_evaluations=5000,
                     mutation=Polynomial(probability=1.0 / problem.number_of_variables, distribution_index=20),
                     crossover=SBX(probability=1.0, distribution_index=20),
                     selection=BinaryTournamentSelection(comparator=RankingAndCrowdingDistanceComparator())
                 ),
                 label='NSGAII with Polynomial Mutation',
                 run=run
             )
         )

 return jobs

if __name__ == '__main__':
 jobs = configure_experiment(problems=[ZDT1(), ZDT2()], n_run=3)

 test = Experiment(
     base_directory='./experiment',
     jobs=jobs
 )
 test.run()

 metrics = [HyperVolume(reference_point=[1, 1]), ComputingTime()]
 data = test.compute_metrics(metrics)
 print(data)